In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy

print(tf.__version__)

2.7.0


In [41]:
magnitude = 10
nb_train_numbers = 6000000
nb_test_numbers = 1000000

In [42]:
%%time
train_numbers = np.random.randint(10**magnitude, size=nb_train_numbers, dtype=np.int64)

Wall time: 122 ms


In [43]:
%%time
test_numbers = np.random.randint(10**magnitude, size=nb_test_numbers, dtype=np.int64)

Wall time: 24.8 ms


In [44]:
%%time
train_number_labels = np.array([sympy.isprime(x) for x in train_numbers])

Wall time: 19.7 s


In [45]:
%%time
test_numbers_levels = np.array([sympy.isprime(x) for x in test_numbers])

Wall time: 3.31 s


In [46]:
def transform_number_to_array(number):
    ar = np.zeros((magnitude, 10))
    nb_str = str(number)
    n = len(nb_str)
    for pos in reversed(range(len(nb_str))):
        digit = int(nb_str[pos])
        ar[pos, digit] = 1
    return ar

In [47]:
transform_number_to_array(1050)

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [48]:
train_numbers_ar = np.array([transform_number_to_array(x) for x in train_numbers])

In [49]:
train_numbers_ar.shape

(6000000, 10, 10)

In [50]:
test_numbers_ar = np.array([transform_number_to_array(x) for x in test_numbers])

In [51]:
test_numbers_ar.shape

(1000000, 10, 10)

In [52]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(magnitude, 10)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [53]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [54]:
model.fit(train_numbers_ar, train_number_labels, epochs=10)

Epoch 1/10
187500/187500 [==============================] - 220s 1ms/step - loss: 0.1429 - accuracy: 0.9545
Epoch 2/10
187500/187500 [==============================] - 221s 1ms/step - loss: 0.1421 - accuracy: 0.9545
Epoch 3/10
187500/187500 [==============================] - 220s 1ms/step - loss: 0.1421 - accuracy: 0.9545
Epoch 4/10
187500/187500 [==============================] - 219s 1ms/step - loss: 0.1421 - accuracy: 0.9545
Epoch 5/10
187500/187500 [==============================] - 211s 1ms/step - loss: 0.1421 - accuracy: 0.9545
Epoch 6/10
187500/187500 [==============================] - 212s 1ms/step - loss: 0.1421 - accuracy: 0.95450s - loss: 0.1421 - 
Epoch 7/10
187500/187500 [==============================] - 212s 1ms/step - loss: 0.1421 - accuracy: 0.9545
Epoch 8/10
187500/187500 [==============================] - 211s 1ms/step - loss: 0.1421 - accuracy: 0.9545
Epoch 9/10
187500/187500 [==============================] - 212s 1ms/step - loss: 0.1420 - accuracy: 0.9545
Epoch 10

In [55]:
test_loss, test_acc = model.evaluate(test_numbers_ar,  test_numbers_levels, verbose=2)

31250/31250 - 23s - loss: 0.1413 - accuracy: 0.9548 - 23s/epoch - 732us/step


In [56]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [57]:
predictions = probability_model.predict(test_numbers_ar)

In [58]:
for i in range(10):
    nb = test_numbers[i]
    prob = predictions[i]
    print("{}, {}, {:.0%}".format(sympy.isprime(nb), nb, prob[1]))
    

False, 8708007948, 0%
False, 2273592062, 0%
False, 1042328794, 0%
False, 5984651675, 0%
False, 4942269578, 0%
True, 9236999867, 12%
False, 5623265271, 12%
False, 3743337360, 0%
False, 5555861211, 12%
False, 3058516765, 0%


In [59]:
nb_tot = 0
nb_primes = 0
nb_corrects = 0
for i, p in enumerate(predictions):
    nb_tot += 1
    if sympy.isprime(test_numbers[i]) == 1:
        nb_primes += 1
        if p[1] > 0.5:
            nb_corrects += 1
print(nb_tot)
print(nb_primes)
print(nb_corrects)

1000000
45198
0
